In [1]:
from urllib.request import urlretrieve, urlopen
import urllib
from bs4 import BeautifulSoup
import csv
import re, string
from itertools import islice
import pickle

In [ ]:
recent_years = {2015, 2014, 2013, 2012}

count = 0
for m in movies:
    if movies[m][1] in recent_years:        
        recent_movies[m] = movies[m]
        

In [2]:
def prepare_datasets(): 
    f = open("recent_movies.pickle", 'rb')
    recent_movies = pickle.load(f)
    del(f)

    f = open("recent_reviews.pickle", 'rb')
    recent_reviews = pickle.load(f)
    del(f)
    
    return [recent_movies, recent_reviews]

recent_movies, recent_reviews = prepare_datasets()
print(len(recent_movies))
print(len(recent_reviews))

3862
3772


In [3]:
# Mad Max: Fury Road
recent_reviews[122882]['A.O. Scott']

['A.O. Scott',
 'New York Times',
 None,
 'http://www.nytimes.com/2015/05/15/movies/review-mad-max-fury-road-still-angry-after-all-these-years.html?partner=rss&emc=rss']

In [4]:
len(recent_movies)

3862

In [5]:
def chunks(data, SIZE):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k for k in islice(it, SIZE)}

recent_movies_chunks = chunks(recent_movies, SIZE=5)
recent_movies_chunks = [m for m in recent_movies_chunks]

print(sum([len(m) for m in recent_movies_chunks]))

type(recent_movies_chunks[-1])

3862


set

In [6]:
rt_url = "http://www.rottentomatoes.com"
rt_search_url = rt_url + "/search/?search="

In [7]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

print(is_ascii('abc'))
print(is_ascii('à'))

def encode_unicode(title):
    """
    replaces unicode characters with percent encodings.
    """
    tmp = ''
    for i in range(0, len(title)):
        c = title[i]
        if is_ascii(c):
            tmp += c
        else:
            tmp += urllib.parse.quote(c)
    tmp = tmp.encode("utf-8")
    return urllib.parse.quote(tmp)
        
print(encode_unicode("ab cå"))

True
False
ab%20c%25C3%25A5


In [8]:
exclude = set(string.punctuation.replace('(','').replace(')', '').replace("'",''))

def get_search_url(movie):
    """
    Finds the rotten tomatoes URL for the given title.
    """
    title = ''.join(ch for ch in movie[0] if ch not in exclude).lower()
    
    if not is_ascii(title):
        # Percent encode UNICODE characters
        title = encode_unicode(title)
        
    search_url = rt_search_url + '+'.join(title.split(' '))
    
    return search_url

print(get_search_url(recent_movies[122882]))
print(get_search_url(recent_movies[108787]))

http://www.rottentomatoes.com/search/?search=mad+max+fury+road
http://www.rottentomatoes.com/search/?search=tom%20at%20the%20farm%20%28tom%20%25C3%25A0%20la%20ferme%29


In [9]:
def convert_letter_grade(score):
    """
    Converts letter grade to score.
    """
    score = score.lower()
    
    if score == 'a':
        return 1
    
    letter = score[0]
    sign = None
    if len(score) > 1:
        sign = score[1]
    
    s = None
    if letter == 'a':
        s = .80
    elif letter == 'b':
        s = .60
    elif letter == 'c':
        s = .40
    elif letter == 'd':
        s = .20
    elif letter == 'f':
        s = 0
        
    if sign == '+':
        s += .1
    elif sign == '-':
        s -= .1
        
    return s

def compute_score(score):
    """
    Takes a score from rotten tomatoes and converts it to a score between 0 and 1. 0 being terrible
    and 1 being perfect. If there isn't a score, returns -1.
    """
    if not score:
        return None
    
    if '/' in score:
        num, denom = score.split('/')
        return float(num)/float(denom)
    
    try:
        return convert_letter_grade(score)
    except:
        return score

print(compute_score("3/4"))
print(compute_score("b+"))

0.75
0.7


In [10]:
def process_review(review):
    reviewer_and_source = review.find("div", {"class" : "critic_name"})
    link_and_score = review.find("div", {"class" : "small subtle"})
    
    source = reviewer_and_source.find('em').getText()
    reviewer = reviewer_and_source.find('a')
    
    if not reviewer:
        reviewer = source
    else:
        reviewer = reviewer.getText()

    review_link = link_and_score.find('a')
    if (review_link):
        review_link = review_link['href']

    score = link_and_score.getText().split('Original Score:')
    if len(score) > 1:
        score = score[1].strip()
    else:
        score = None
    
    return [reviewer, source, compute_score(score), review_link]
    
    
page_suffix = "?page=%d&sort="
def get_reviews(url):
    """
    Given an RT reviews page, this grabs links to all of the reviews on the page.
    """
    if not url:
        return {}
    
    if not (url[-1] == '/'):
        url += '/'
    
    all_reviews = {}
    for i in range(1,20):
        page = url + (page_suffix % i)
        
        try:
            response = urlopen(page)
        except:
            break
        
        soup = BeautifulSoup(response)
        reviews = soup.findAll("div", { "class" : "row review_table_row" })
    
        for review in reviews:
            data = process_review(review)
            all_reviews[data[0]] = data
    return all_reviews

toy_story_reviews = get_reviews("http://www.rottentomatoes.com/m/toy_story/reviews")
print(len(toy_story_reviews))

77


In [11]:
def get_movie_url(movie):
    """
    Given a movie, determines the rotten tomatoes URL for the critics' reviews of that movie.
    """
    search_url = get_search_url(movie)
    
    response = urlopen(search_url)
    soup = BeautifulSoup(response)
    
    url = None
    
    # No results tag
    if (soup.find("h1", {"class" : "center noresults"})):
        return url
    
    if "search results" in soup.find("title").text.lower():
        # Find the movie with the correct year
        year = movie[1]
        results = soup.findAll("li", {"class" : "media bottom_divider clearfix"})
        
        for i in results:
            result_year = i.find("span", {"class" : "movie_year"}).getText().strip()[1:5]
            if not result_year:
                continue
            result_year = int(result_year)
            result_url = rt_url + i.find("a")["href"]
            
            title = i.find("div", {"class" : "nomargin media-heading bold"}).find("a").getText().lower()

            if year == result_year:
                url = result_url
                break
    
    else:
        url = response.url.split('?')[0]

    if url:
        url = url + "reviews"
    
    return url


# The battery, which isn't on rotten tomatoes
print(get_movie_url(recent_movies[103816]))

# Should be mad max
print(get_movie_url(recent_movies[122882]))

# Foreign film with unicode
print(get_movie_url(recent_movies[108787]))


None
http://www.rottentomatoes.com/m/mad_max_fury_road/reviews
http://www.rottentomatoes.com/m/tom_at_the_farm/reviews


In [12]:
len(recent_reviews)

3772

In [13]:
folder = "recent_reviews/"

def pickle_reviews(mid, rs):
    """
    dump pickle of reviews to disk. Super inefficient but I really don't want to lose this data. 
    Would be better to setup SQL database but I'm cheap.
    """
    f = open("%s%d.pickle" % (folder, mid), 'wb')
    pickle.dump(rs, f)

In [25]:
error_url = []
error_rev = []

In [142]:
count = 0

len(recent_movies_chunks)

# This obviously takes a really long time to do all chunks
for i in range(0,0):
    count += 1
    if (count % 10 == 0):
        print(count)
    cur = recent_movies_chunks[i]
    
    for mid in cur:
        
        if mid in recent_reviews:
            continue
            
        movie = recent_movies[mid]
        
        if len(movie) > 3:
            url = movie[3]
        else:
            try:
                url = get_movie_url(movie)
                movie.append(url)
            except Exception as ex:
                error_url.append([mid, movie, ex])
                continue
                
        try:
            rs = get_reviews(url)
        except Exception as ex:
            error_rev.append([mid, movie, ex])
            continue
    
        recent_reviews[mid] = rs
        pickle_reviews(mid, rs)
        
        
    f = open("recent_movies.pickle", 'wb')
    pickle.dump(recent_movies, f)
    f = open("recent_reviews.pickle", 'wb')
    pickle.dump(recent_reviews, f)

In [141]:
count = 0
for i in recent_reviews:
    if len(recent_reviews[i]) > 0:
        count += len(recent_reviews[i])
count

123117

In [144]:
f = open("error_url.pickle", 'wb')
pickle.dump(error_url, f)

f = open("error_rev.pickle", 'wb')
pickle.dump(error_rev, f)

'Susan Granger'

In [16]:
reviewers = {}

In [17]:
for i in recent_reviews:
    reviews = recent_reviews[i]
    title, year, genre, url = recent_movies[i]

    for name in reviews:

        if name not in reviewers:
            reviewers[name] = {}
        
        rev, source, score, link = reviews[name]
        reviewers[name][i] = [title, year, genre, source, score, url, link]


In [18]:
len(recent_reviews)

3772